In [11]:
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from matplotlib import pyplot as plt

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

import bo_methods_lib
from bo_methods_lib.bo_functions_generic import gen_theta_set

from bo_methods_lib.CS2_bo_plotters import plot_obj_abs_min, value_plotter, plot_obj, plot_Theta, plot_Theta_min,path_name, plot_org_train, save_fig, plot_EI_abs_max

from bo_methods_lib.CS2_bo_functions_multi_dim import set_ep

In [12]:
#https://numpy.org/doc/stable/reference/generated/numpy.loadtxt.html

def csv_to_array(csv_path):
    """
    Turn a csv file into a numpy array so that it can be used with bo_plotters
    
    Parameters:
    -----------
        csv_path: str, path of csv you want to turn into an array
    Returns:
    --------
        csv_array: np.ndarray, array on values in the CSV file
    """
    
    csv_pd = pd.read_csv(csv_path, index_col = False)
    csv_pd.drop(columns=csv_pd.columns[0], 
        axis=1, 
        inplace=True)
    csv_array = csv_pd.to_numpy()
    
    return csv_array

In [13]:
# DateTime = "2022/10/21/13-05" #Date and Time - Dispayed in job file
DateTime = "2022/10/21/13-18" #Date and Time - Dispayed in job file
# DateTime = None

emulator = True
obj = "obj"
sparse_grid = True
sep_fact = 0.1

exp_d = 1
n = 5

param_dict = {0 : '\\theta_1', 1 : '\\theta_2'}
indecies = [list(param_dict.keys())[list(param_dict.values()).index('\\theta_1')],
            list(param_dict.keys())[list(param_dict.values()).index('\\theta_2')]]

Theta_True = np.array([1,-1])

bounds = np.array([[-2, -2],
                   [ 2,  2]])


# exp_d = 2
# n = 15 #Number of experimental data points to use

# param_dict = {0 : 'a_1', 1 : 'a_2', 2 : 'a_3', 3 : 'a_4',
#               4 : 'b_1', 5 : 'b_2', 6 : 'b_3', 7 : 'b_4'}
# indecies = [list(param_dict.keys())[list(param_dict.values()).index('a_1')],
#             list(param_dict.keys())[list(param_dict.values()).index('a_2')]]

# Constants = np.array([[-200,-100,-170,15],
#                       [-1,-1,-6.5,0.7],
#                       [0,0,11,0.6],
#                       [-10,-10,-6.5,0.7],
#                       [1,0,-0.5,-1],
#                       [0,0.5,1.5,1]])

# Theta_True = Constants[1:3].flatten()

# bounds = np.array([[-2, -2, -10, -2, -2, -2,  5, -2],
#                    [ 2,  2,   0,  2,  2,  2, 15,  2]])

param_names_list = [param_dict[0], param_dict[1]] #Fill in depending on problem
mesh_combo = str(param_names_list[0]) + "-" + str(param_names_list[1])

mesh_combo = None
# run = 1
# Bo_iter = 1

run = 12
Bo_iter = 6

set_lengthscale = None
save_CSV = False

t = 20
if emulator == True:
    t = n*t

#Pull Experimental data from CSV
exp_data_doc = '../Input_CSVs/Exp_Data/d='+str(exp_d)+'/n='+str(n)+'.csv'
exp_data = np.array(pd.read_csv(exp_data_doc, header=0,sep=","))
Xexp = exp_data[:,1:exp_d+1]
Yexp = exp_data[:,-1]

#Define GP Testing space
p=20
d = len(Theta_True)


# Need to redefine to save theta_set_org 
# theta_set_org = gen_theta_set(LHS = True, n_points = p, dimensions = d, bounds = bounds)
# Theta1_lin = np.linspace(np.min(theta_set_org[:,indecies[0]]),np.max(theta_set_org[:,indecies[0]]), p)
# Theta2_lin = np.linspace(np.min(theta_set_org[:,indecies[1]]),np.max(theta_set_org[:,indecies[1]]), p)
# theta_mesh = np.array(np.meshgrid(Theta1_lin, Theta2_lin)) 

theta_mesh = gen_theta_set(LHS = False, n_points = p, dimensions = d, bounds = bounds)
theta_mesh = theta_mesh.T.reshape((2, p, p))
# print(theta_mesh.shape)
    
ep = set_ep(emulator, obj, sparse_grid)
# ep = 1

# tot_iter = bo_iters= 3 #Total Number of iters - Dispayed in job file
# tot_runs = 2 #Total Number of runs - Dispayed in job file

tot_iter = bo_iters= 100 #Total Number of iters - Dispayed in job file
tot_runs = 15 #Total Number of runs - Dispayed in job file

is_figure = False
save_figure = False

run -= 1 # run should be number you want -1
Bo_iter -= 1 #Iter you want -1

In [14]:
#WORKS
array_info = []
sep_fact_list = np.linspace(0.1,1,10)
for k in range(len(sep_fact_list)):     
    csv_end = ".csv" #For obj
    fxn_obj = "plot_obj"
    fxn_obj_min = "plot_obj_abs_min"
    path_obj = "../" + path_name(emulator, ep, sparse_grid, fxn_obj, set_lengthscale, t, obj, mesh_combo = None, 
                                 bo_iter= None, title_save = None, run = None, tot_iter=tot_iter, tot_runs=tot_runs, 
                                 DateTime = DateTime, sep_fact = sep_fact_list[k], is_figure = is_figure, csv_end = ".csv")
    path_obj_min = "../" + path_name(emulator, ep, sparse_grid, fxn_obj_min, set_lengthscale, t, obj, mesh_combo = None, 
                                 bo_iter= None, title_save = None, run = None, tot_iter=tot_iter, tot_runs=tot_runs, 
                                 DateTime = DateTime, sep_fact = sep_fact_list[k], is_figure = is_figure, csv_end = ".csv")
    obj_array = csv_to_array(path_obj)
    obj_array_min = csv_to_array(path_obj_min)
    
    for i in range(tot_runs): 
        obj_df_run = pd.DataFrame(data = obj_array[i])
        obj_df_i = obj_df_run.loc[(abs(obj_df_run) > 1e-6).any(axis=1),0]
        max_iter = len(obj_df_i)
        obj_no_zero_arr = list(np.array(obj_df_i).flatten())
        
        obj_df_run_min = pd.DataFrame(data = obj_array_min[i])
        obj_df_i_min = obj_df_run_min.loc[(abs(obj_df_run_min) > 1e-6).any(axis=1),0]
        max_iter_min = len(obj_df_i_min)
        obj_no_zero_arr = list(np.array(obj_df_i_min).flatten())
        for j in range(len(obj_df_i)):
            array_info.append([np.round(sep_fact_list[k],1), i+1, j+1,np.exp(obj_df_i[j]), len(obj_df_i), 
                               np.exp(obj_df_i_min[j])])

df = pd.DataFrame(data = array_info, columns = ["Separation Factor", "Run", "Iteration", "SSE", "Max Eval", "Minimum SSE"])

for k in range(len(df)):  
    for i in range(len(param_dict.keys())):
        fxn_theta = "plot_Theta"
        csv_end = "/"+param_dict[i]+ ".csv" #For Thetas #Need to ensure this saves correctly

        path_i = "../" + path_name(emulator, ep, sparse_grid, fxn_theta, set_lengthscale, t, obj, mesh_combo = None, bo_iter= None, 
                  title_save = None, run = None, tot_iter=tot_iter, tot_runs=tot_runs, DateTime = DateTime, 
                         sep_fact = df["Separation Factor"][k], is_figure = is_figure, csv_end = csv_end)
        path_i = path_i.replace("Param_Conv/\\theta_"+str(i+1), "Theta_Conv/Theta_Conv_"+str(i+1))
        theta_i = csv_to_array(path_i)
#         print(df["Run"][k], df["Iteration"][k])git 
        theta_ij = theta_i[df["Iteration"][k]-1, df["Run"][k]-1]
        df.loc[df.index[k], "Theta " + str(i+1)] = theta_ij

        path_i_min = path_i.replace("Param_Conv/\\theta_"+str(i+1), "Theta_Conv_min/Theta_Conv_min"+str(i+1))
        theta_i_min = csv_to_array(path_i_min)
#         print(df["Run"][k], df["Iteration"][k])
        theta_ij_min = theta_i_min[df["Iteration"][k]-1, df["Run"][k]-1]
        df.loc[df.index[k], "Min Theta " + str(i+1)] = theta_ij_min
#         Theta_info = theta_i[]

print(df[20:40])

    Separation Factor  Run  Iteration         SSE  Max Eval  Minimum SSE  \
20                0.1   11          1    2.564324         2     2.564324   
21                0.1   11          2   46.912139         2     2.564324   
22                0.1   12          1   15.870302         5    15.870302   
23                0.1   12          2    8.542752         5     8.542752   
24                0.1   12          3    7.123337         5     7.123337   
25                0.1   12          4    5.269469         5     5.269469   
26                0.1   12          5    4.723241         5     4.723241   
27                0.1   13          1    8.432739         2     8.432739   
28                0.1   13          2   11.132078         2     8.432739   
29                0.1   14          1   50.946648         2    50.946648   
30                0.1   14          2  100.245072         2    50.946648   
31                0.1   15          1   46.257240         2    46.257240   
32          

In [17]:
#WORKS
df = pd.DataFrame()


sep_fact_list = np.linspace(0.1,1,10)
for k in range(len(sep_fact_list)): 
    
    csv_end = ".csv" #For obj
    fxn_obj = "plot_obj"
    path_obj = "../" + path_name(emulator, ep, sparse_grid, fxn_obj, set_lengthscale, t, obj, mesh_combo = None, 
                                 bo_iter= None, title_save = None, run = None, tot_iter=tot_iter, tot_runs=tot_runs, 
                                 DateTime = DateTime, sep_fact = sep_fact_list[k], is_figure = is_figure, csv_end = ".csv")
    # print(path)
    obj_array = csv_to_array(path_obj)
    all_obj_list = []
    for i in range(tot_runs): 
        #Plot data
        obj_df_run = pd.DataFrame(data = obj_array[i])
        print(obj_df_run)
#         obj_df_i = obj_df_run.loc[(abs(obj_df_run) > 1e-6).any(axis=1),0]
#         obj_no_zero_arr = list(np.array(obj_df_i).flatten())
#         all_obj_list += obj_no_zero_arr
#         df["Separation Factor"] = sep_fact_list[k]
#         df["Run"] = int(i)
#         df["SSE"] = obj_df_run
#     print(all_obj_list[0:5])
# print(df)

           0
0   2.516077
1   0.382030
2   0.000000
3   0.000000
4   0.000000
..       ...
95  0.000000
96  0.000000
97  0.000000
98  0.000000
99  0.000000

[100 rows x 1 columns]
           0
0   0.945944
1  -0.234171
2   0.000000
3   0.000000
4   0.000000
..       ...
95  0.000000
96  0.000000
97  0.000000
98  0.000000
99  0.000000

[100 rows x 1 columns]
           0
0   1.221619
1   1.711095
2   0.000000
3   0.000000
4   0.000000
..       ...
95  0.000000
96  0.000000
97  0.000000
98  0.000000
99  0.000000

[100 rows x 1 columns]
           0
0   2.537995
1   1.830525
2   0.000000
3   0.000000
4   0.000000
..       ...
95  0.000000
96  0.000000
97  0.000000
98  0.000000
99  0.000000

[100 rows x 1 columns]
           0
0   0.731942
1   0.811213
2   0.000000
3   0.000000
4   0.000000
..       ...
95  0.000000
96  0.000000
97  0.000000
98  0.000000
99  0.000000

[100 rows x 1 columns]
           0
0   4.787673
1   4.020429
2   0.000000
3   0.000000
4   0.000000
..       ...
95  0.000

           0
0  -0.701039
1  -1.568814
2  -4.462221
3  -3.183020
4  -6.063335
..       ...
95  0.000000
96  0.000000
97  0.000000
98  0.000000
99  0.000000

[100 rows x 1 columns]
           0
0  -0.701039
1  -1.569047
2  -4.345085
3  -5.934584
4  -5.541114
..       ...
95  0.000000
96  0.000000
97  0.000000
98  0.000000
99  0.000000

[100 rows x 1 columns]
           0
0  -0.701039
1  -1.568813
2  -4.290003
3  -6.008655
4  -5.584465
..       ...
95  0.000000
96  0.000000
97  0.000000
98  0.000000
99  0.000000

[100 rows x 1 columns]
           0
0  -0.701039
1  -1.568815
2  -4.535578
3  -3.232919
4   0.000000
..       ...
95  0.000000
96  0.000000
97  0.000000
98  0.000000
99  0.000000

[100 rows x 1 columns]
           0
0  -0.701039
1  -1.568814
2  -4.352865
3  -6.108158
4  -5.965429
..       ...
95  0.000000
96  0.000000
97  0.000000
98  0.000000
99  0.000000

[100 rows x 1 columns]
           0
0  -0.701039
1  -1.568813
2  -4.535574
3  -3.232918
4   0.000000
..       ...
95  0.000

In [8]:
#WORKS

sep_fact_list = np.linspace(0.1,1,10)
med_iter_list = []
sse_best_list = []
iter_best_num_list = []
iter_median_num_list = []
run_best_num_list = []
median_obj_min =[]
for k in range(len(sep_fact_list)):  
    csv_end = ".csv" #For obj
    fxn = "plot_obj"

    path = "../"+ path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj, mesh_combo = None, bo_iter= None, 
              title_save = None, run = None, tot_iter=tot_iter, tot_runs=tot_runs, DateTime = DateTime, 
                     sep_fact = sep_fact_list[k], is_figure = is_figure, csv_end = ".csv")
    # print(path)
    obj_abs_min = csv_to_array(path)

    iter_max = []
    run_best = []
    iter_best = []
    sse_best = []
    for i in range(tot_runs):
        obj_min_df_run = pd.DataFrame(data = obj_abs_min[i])
        obj_min_df_i = obj_min_df_run.loc[(abs(obj_min_df_run) > 1e-6).any(axis=1),0]
#         print(obj_min_df_i)
        obj_min_no_zero_arr = list(np.array(obj_min_df_i).flatten())
        median_obj_min.append( np.min(obj_min_no_zero_arr) )
#         print(np.min(obj_min_no_zero_arr))
        sse_best.append(np.amin(obj_min_df_i))
        iter_max.append(len(obj_min_df_i))
        min_iter = np.where(np.amin(obj_min_df_i) == obj_min_df_i)
        #Not sure how to fix but sparse grid thetas are weird because the first theta is 0 for some reason for runs 5,6,7
#         if sparse_grid == True:
#             if i ==4 or i==5 or i == 6:
#                 iter_best.append(min_iter[0][0]+1)
#             else:
        iter_best.append(min_iter[0][0])
        run_best.append(i)
#     print(iter_max)    
    iter_median_num_list.append(np.median(iter_max))
    iter_best_act = np.array(iter_best) + 1
    run_best_act = np.array(run_best) + 1 
    sse_best_num = np.amin(sse_best)
    sse_best_list.append(sse_best_num)
    iter_best_ind = sse_best.index(sse_best_num)
    run_best_list = run_best_act[iter_best_ind]
    iter_best_num = iter_best_act[iter_best_ind]
    median_iters = np.median(iter_best_act)
    med_iter_list.append(median_iters)
    iter_best_num_list.append(iter_best_num)
    run_best_num_list.append(run_best_list)
# print(len(median_obj_min))
# print(median_obj_min)
median_obj_min_num = np.median(np.array(median_obj_min))
    
print(med_iter_list)
print(sse_best_list)
print(iter_best_num_list)
print(iter_median_num_list)
print(run_best_num_list)
print(np.exp(median_obj_min_num))
    

[2.0, 2.0, 1.0, 1.0, 2.0, 7.0, 9.0, 6.0, 7.0, 4.0]
[-0.2341707708620758, -0.0523112587139613, -0.6789879626185658, -1.8675058346360105, -6.088281211816648, -6.304517222032395, -7.6184965485367, -5.9139624734051495, -5.77414943387773, -6.10815805076885]
[2, 1, 1, 2, 10, 10, 6, 6, 9, 4]
[2.0, 2.0, 2.0, 2.0, 2.0, 9.0, 10.0, 7.0, 8.0, 5.0]
[2, 5, 5, 7, 4, 15, 12, 1, 13, 8]
0.308946231271654


In [9]:
sep_fact_list = np.linspace(0.1,1,10)
theta_Opt = np.zeros((len(sep_fact_list),d))

for k in range(len(sep_fact_list)):
    Theta_array = []
    fxn = "plot_Theta"
    for i in range(len(param_dict.keys())):
        csv_end = "/"+param_dict[i]+ ".csv" #For Thetas #Need to ensure this saves correctly

        path_i = "../" + path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj, mesh_combo = None, bo_iter= None, 
                  title_save = None, run = None, tot_iter=tot_iter, tot_runs=tot_runs, DateTime = DateTime, 
                         sep_fact = sep_fact_list[k], is_figure = is_figure, csv_end = csv_end)
        path_i = path_i.replace("Param_Conv/\\theta_"+str(i+1), "Theta_Conv/Theta_Conv_"+str(i+1))
#         print(path_i)
        Theta_array.append( csv_to_array(path_i) )

    Theta_array = np.array(Theta_array)
    #Not sure how to fix but sparse grid thetas are weird because of mismatched 0 columns/rows for k>5
    Theat_Opt = Theta_array[:, iter_best_num_list[k]-1,  run_best_num_list[k]-1]
    theta_Opt[k] = Theat_Opt
print(theta_Opt)

[[ 1.19204873 -0.89500599]
 [ 0.7608077  -0.89418591]
 [ 0.77076789 -1.0193174 ]
 [ 0.89446011 -1.04369145]
 [ 0.99179447 -0.99531665]
 [ 0.99384819 -0.99607024]
 [ 0.99502016 -0.99969762]
 [ 0.9927136  -0.99450618]
 [ 0.99193723 -0.9938964 ]
 [ 0.99111488 -0.99546377]]


In [10]:
#Create table arrays using pandas
df = pd.DataFrame(list(zip(theta_Opt[:,0], theta_Opt[:,1], sse_best_list, median_obj,iter_best_num_list,iter_median_num_list)),
               columns =['Theta 1', 'Theta 2', 'Best SSE', "Median SSE", "Best Eval #", "Median Eval #"])

In [11]:
print(df)

    Theta 1   Theta 2  Best SSE  Median SSE  Best Eval #  Median Eval #
0  0.994778 -1.004316 -9.601840   -0.068837           45          100.0
1  0.347286 -0.942790 -0.693763    1.071300            8            4.0
2  0.948718 -0.904418 -3.858616    2.710024           28            3.0
3  0.000000  0.000000  1.459963    2.387560          100            5.0
4  0.000000  0.000000  1.597998    2.460672          100            6.0
5  0.000000  0.000000  0.312656    2.478331          100           19.0
6  0.000000  0.000000  1.484999    2.493710            1          100.0
7  0.000000  0.000000  2.330423    2.613351          100          100.0
8  1.131402 -1.617355  2.347631    2.495912          100          100.0
9  1.042000 -1.577675  2.398091    2.415741          100          100.0
